In [ ]:
# Coder_Hussam Qassim

# Import the necessary libraries 
import tensorflow as tf 
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data

# Define the NN parameters
n_inputs = 28*28 # MNIST
n_hidden1 = 300 
n_hidden2 = 100
n_outputs = 10 

# Create the inpute and lable data 
x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

# Create the neuron function 
def neuron_layer(x, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(x.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        w = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(x, w) + b
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z

# Create the graph on 1 input layer and 2 hidden layers and one output layer 
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(x, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")  
    
# Create the cost function     
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

# Craete the optimizer 
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op	= optimizer.minimize(loss)
    
# Evaluate the NN     
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
# Initialize the variables     
init = tf.global_variables_initializer()

# Initialize the saver for save the model
saver = tf.train.Saver()

# Fetch the data 
mnist = input_data.read_data_sets("data/")

# define the number of the epochs and the size of the batch
n_epochs = 400
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
            acc_train = accuracy.eval(feed_dict={x: x_batch, y: y_batch})
            acc_test = accuracy.eval(feed_dict={x: mnist.test.images,
                                                y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "data/my_model_final.ckpt")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
